# Seq2Seq model without using legacy stuff

In [1]:
%%bash
python -m spacy download en
python -m spacy download de

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907055 sha256=24cf6620b0a9cc6974c5478ff58321eb5e13b32c34bdd234093736ab9bdff4bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-az5l7ic3/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [2]:
!pip install torchtext

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

import spacy
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import random
import math
import time

In [4]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

In [6]:
# Create source and target language tokenizer. Make sure to install the dependencies.
token_transform_de = get_tokenizer('spacy', language='de')
token_transform_en = get_tokenizer('spacy', language='en')

In [7]:
# helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    if language == SRC_LANGUAGE:
        for src, _ in data_iter:
            yield token_transform_de(src)
    elif language == TGT_LANGUAGE:
        for _ , tgt in data_iter:
            yield token_transform_en(tgt)

In [8]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [9]:
# Training and Testing data Iterator 
train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 970kB/s]
mmt16_task1_test.tar.gz: 100%|██████████| 43.9k/43.9k [00:00<00:00, 155kB/s]


In [10]:
# check the split size
len(train_iter), len(test_iter)

(29000, 1000)

In [11]:
# test iterator
next(train_iter)

('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n',
 'Two young, White males are outside near many bushes.\n')

In [12]:
# Create torchtext's train Vocab object 
train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform_trn_src = build_vocab_from_iterator(yield_tokens(train_iter, SRC_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_trn_src.set_default_index(UNK_IDX)

train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform_trn_tgt = build_vocab_from_iterator(yield_tokens(train_iter, TGT_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_trn_tgt.set_default_index(UNK_IDX)

In [13]:
vocab_transform_trn_src(['Zwei','junge', 'weiße','Männer', 'sachin'])

[22, 86, 258, 32, 0]

In [14]:
vocab_transform_trn_tgt(['two', 'young'])

[67, 26]

In [15]:
# Create torchtext's test Vocab object 
train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform_tst_src = build_vocab_from_iterator(yield_tokens(train_iter, SRC_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_tst_src.set_default_index(UNK_IDX)

train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform_tst_tgt = build_vocab_from_iterator(yield_tokens(train_iter, TGT_LANGUAGE),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)
vocab_transform_tst_tgt.set_default_index(UNK_IDX)

In [16]:
# helper fucntions for transformations 

from torch.nn.utils.rnn import pad_sequence

# helper function to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), 
                      torch.tensor(token_ids), 
                      torch.tensor([EOS_IDX])))

# src and tgt language text transforms to convert raw strings into tensors indices
text_transform_trn = {}
text_transform_trn[SRC_LANGUAGE] = sequential_transforms(token_transform_de, #Tokenization
                                            vocab_transform_trn_src, #Numericalization
                                            tensor_transform) # Add BOS/EOS and create tensor

text_transform_trn[TGT_LANGUAGE] = sequential_transforms(token_transform_en, #Tokenization
                                            vocab_transform_trn_tgt, #Numericalization
                                            tensor_transform) # Add BOS/EOS and create tensor

text_transform_tst = {}
text_transform_tst[SRC_LANGUAGE] = sequential_transforms(token_transform_de, #Tokenization
                                            vocab_transform_tst_src, #Numericalization
                                            tensor_transform) # Add BOS/EOS and create tensor

text_transform_tst[TGT_LANGUAGE] = sequential_transforms(token_transform_en, #Tokenization
                                            vocab_transform_tst_tgt, #Numericalization
                                            tensor_transform) # Add BOS/EOS and create tensor


# function to collate data samples into batch tesors
def collate_fn_trn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform_trn[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform_trn[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

def collate_fn_tst(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform_tst[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform_tst[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [17]:
# Test Data Loaders
from torch.utils.data import DataLoader
BATCH_SIZE = 128

# train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

# train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_trn)
# test_dataloader = DataLoader(test_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_tst)

# Building the Seq2Seq Model

## Encoder

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        
        self.embedding = nn.Embedding(input_dim, emb_dim) #no dropout as only one layer!
        
        self.rnn = nn.GRU(emb_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded) #no cell state!
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden

## Decoder

In [19]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
        
        self.fc_out = nn.Linear(emb_dim + hid_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, context):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #context = [n layers * n directions, batch size, hid dim]
        
        #n layers and n directions in the decoder will both always be 1, therefore:
        #hidden = [1, batch size, hid dim]
        #context = [1, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        emb_con = torch.cat((embedded, context), dim = 2)
            
        #emb_con = [1, batch size, emb dim + hid dim]
            
        output, hidden = self.rnn(emb_con, hidden)
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        #output = [batch size, emb dim + hid dim * 2]
        
        prediction = self.fc_out(output)
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden

## Seq2Seq Model

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is the context
        context = self.encoder(src)
        
        #context also used as the initial hidden state of the decoder
        hidden = context
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and the context state
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, context)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

In [21]:
INPUT_DIM = len(vocab_transform_trn_src)
OUTPUT_DIM = len(vocab_transform_trn_tgt)

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [22]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(19206, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(10840, 256)
    (rnn): GRU(768, 512)
    (fc_out): Linear(in_features=1280, out_features=10840, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [23]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 24,729,688 trainable parameters


In [24]:
optimizer = optim.Adam(model.parameters())

In [25]:
# criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
criterion = nn.CrossEntropyLoss()

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, (src_txt,tgt_txt) in enumerate(iterator):
        
        src = src_txt.to(device)
        tgt = tgt_txt.to(device)
        
        optimizer.zero_grad()
        
        output = model(src, tgt)
        
        #tgt = [tgt len, batch size]
        #output = [tgt len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        tgt = tgt[1:].view(-1)
        
        #tgt = [(tgt len - 1) * batch size]
        #output = [(tgt len - 1) * batch size, output dim]
        
        loss = criterion(output, tgt)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, (src_txt,tgt_txt) in enumerate(iterator):

            src = src_txt.to(device)
            tgt = tgt_txt.to(device)

            output = model(src, tgt, 0) #turn off teacher forcing

            #tgt = [tgt len, batch size]
            #output = [tgt len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            tgt = tgt[1:].view(-1)

            #tgt = [(tgt len - 1) * batch size]
            #output = [(tgt len - 1) * batch size, output dim]

            loss = criterion(output, tgt)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
N_EPOCHS = 10
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()

    train_iter, test_iter = Multi30k(split=('train', 'test'), language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    
    train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_trn)
    test_dataloader = DataLoader(test_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn_tst)

    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if test_loss < best_test_loss:
    #     best_test_loss = test_loss
    #     torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Test. Loss: {test_loss:.3f} |  Test. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 1m 5s
	Train Loss: 2.934 | Train PPL:  18.810
	 Test. Loss: 2.954 |  Test. PPL:  19.176
Epoch: 02 | Time: 1m 5s
	Train Loss: 2.336 | Train PPL:  10.338
	 Test. Loss: 2.803 |  Test. PPL:  16.488
Epoch: 03 | Time: 1m 5s
	Train Loss: 2.158 | Train PPL:   8.655
	 Test. Loss: 2.603 |  Test. PPL:  13.508
Epoch: 04 | Time: 1m 5s
	Train Loss: 2.005 | Train PPL:   7.428
	 Test. Loss: 2.637 |  Test. PPL:  13.972
Epoch: 05 | Time: 1m 5s
	Train Loss: 1.875 | Train PPL:   6.520
	 Test. Loss: 2.428 |  Test. PPL:  11.341
Epoch: 06 | Time: 1m 5s
	Train Loss: 1.743 | Train PPL:   5.716
	 Test. Loss: 2.434 |  Test. PPL:  11.406
Epoch: 07 | Time: 1m 5s
	Train Loss: 1.648 | Train PPL:   5.199
	 Test. Loss: 2.370 |  Test. PPL:  10.702
Epoch: 08 | Time: 1m 5s
	Train Loss: 1.521 | Train PPL:   4.578
	 Test. Loss: 2.427 |  Test. PPL:  11.329
Epoch: 09 | Time: 1m 5s
	Train Loss: 1.438 | Train PPL:   4.214
	 Test. Loss: 2.335 |  Test. PPL:  10.326
Epoch: 10 | Time: 1m 5s
	Train Loss: 1.345 | T